# <center> Naive Bayes Classifier </center>

# Tổng quan

- [1. Giới thiệu](#1.-Giới-thiệu)
- [2. Ý tưởng chính](#2.-Ý-tưởng-chính)
- [3. Các loại phân phối dữ liệu](#3.-Các-loại-phân-phối-dữ-liệu)
    - [3.1 Gaussian Naive Bayes](#3.1-Gaussian-Naive-Bayes)
    - [3.2 Multinomial Naive Bayes](#3.2-Multinomial-Naive-Bayes)
    - [3.3 Bernoulli Naive Bayes](#3.3-Bernoulli-Naive-Bayes)
- [4. Ví dụ phân loại hoa Iris bằng thư viện sklearn](#4.-Ví-dụ-phân-loại-hoa-Iris-bằng-thư-viện-sklearn)
- [5. Kết luận](#5.-Kết-luận)
    - [5.1 Ưu điểm](#5.1-Ưu-điểm)
    - [5.2 Nhược điểm](#5.2-Nhược-điểm)
    - [5.3 Ứng dụng](#5.3-Ứng-dụng)
- [6. Tài liệu tham khảo](#6.-Tài-liệu-tham-khảo)


# 1. Giới thiệu

Trong các phần trước ta đã tìm hiểu qua một trong những mô hình có sử dụng xác suất là **Hồi quy logistic (logistic regression)**. Ở phần này ta cũng tìm hiểu một mô hình có lẽ được sử dụng rộng rãi nhất là **Naive Bayes**.

**Naive Bayes** là một thuật toán phân loại các vấn đề nhị phân (hai lớp) hoặc nhiều lớp. Nó được gọi là **Naive Bayes** hoặc **idiot Bayes** vì các tính toán xác suất cho mỗi lớp được đơn giản hóa để làm cho phép tính có thể thực hiện được.

Thay vì cố gắng tính toán xác suất của từng giá trị thuộc tính, **Naive Bayes** giả định là các đặc trưng của dữ liệu độc lập có điều kiện với giá trị của từng lớp.

Đây là một giả định rất mạnh mà hầu như không có trong dữ liệu thực, tức là các thuộc tính không phụ thuộc với nhau ( _independent variable_ ). Tuy nhiên, cách tiếp cận này lại mang lại hiệu quả đáng kinh ngạc. Giả thiết về sự độc lập của các chiều dữ liệu này được gọi là **Naive Bayes**. Cách xác định lớp của dữ liệu dựa trên giả thiết này có tên là *Naive Bayes Classifier*.

Nó hoạt động khá tốt trong nhiều bài toán thực tế, đặc biệt là trong các bài toán phân loại văn bản.

# 2. Ý tưởng chính

Xét bài toán phân loại với $C$ lớp $c_1,c_2,..$ giả sử có một điểm dữ liệu $x \in \mathbb{R}^d$. Tính xác suất để điểm dữ liệu này rơi vào vào lớp $c$. Chính xác là tính: $$p(y=c|x) \quad (1)$$

Biểu thức này, nếu tính được, sẽ giúp chúng ta xác định được xác suất để điểm dữ liệu rơi vào mỗi lớp. Từ đó có thể giúp xác định lớp của điểm dữ liệu đó bằng cách chọn ra lớp có xác suất cao nhất: $$c=arg \max_{c\in \{c_1,c_2,..\}}p(c|x) \quad (2)$$
Biểu thức $(2)$ thường khó tính được trực tiếp.Thay vào đó ta sử dụng [Định lý Bayes](https://vi.wikipedia.org/wiki/%C4%90%E1%BB%8Bnh_l%C3%BD_Bayes):

$$c=arg \max_{c}p(c|x) \quad (3)$$
$$\Leftrightarrow c=arg \max_{c}\frac{p(x|c)p(c)}{p(x)} \quad (4)$$
$$\Leftrightarrow c=arg \max_{c}p(x|c)p(c) \quad (5)$$

Từ $(3)$ sang $(4)$ ta sử dụng định lý _Bayes_.

Từ $(4)$ sang $(5)$ tại vì mẫu số $p(x)$ không phụ thuộc vào $c$. 

$p(c)$ là xác suất rơi vào lớp $c$, được tính bằng tỉ lệ số điểm dữ liệu trong tập đào tạo rơi vào lớp này chia cho tổng số lượng dữ liệu trong tập đào tạo.

$p(x|c)$ phân phối của các điểm dữ liệu trong lớp $c$, thường rất khó tính toán vì $x$ là một biến ngẫu nhiên nhiều chiều. Để tính toán được đơn giản, giả sử rằng các thành phần của biến ngẫu nhiên $x$ là độc lập với nhau, nếu biết $c$.

$$p(x|c)=p(x_1,x_2,x3_,...x_d|c)=\prod_{i=1}^dp(x_i|c) \quad (6)$$

Bước _training_ , các phân phối $p(c)$ và $p(x_i|c)$ sẽ được xác định dựa vào dữ liệu đào tạo.

Việc tính $p(x_i|c)$ phụ thuộc vào loại dữ liệu, có ba loại được sử dụng phổ biến là: Gaussian Naive Bayes, Multinomial Naive Bayes, và Bernoulli Naive .

Bước thử nghiệm, lớp $c$ của điểm dữ liệu $x$ sẽ được xác định bằng:
$$c=arc \max_{c \in \{c_1,c_2,..\}}p(c)\prod_{i=1}^dp(x_i|c) \quad (7)$$

Khi $d$ lớn và các xác suất nhỏ thì biểu thức $(7)$ sẽ rất nhỏ, khi tính toán dễ xảy ra sai số. Để giải quyết vấn đề này người ta thường lấy $\log$ của vế phải:
$$c=arc \max_{c \in \{c_1,c_2,..\}}\bigg[\log\big(p(c)\big)+\sum_{i=1}^d \log\big(p(x_i|c)\big)\bigg] \quad (7.1)$$

**Lưu ý:** bước quan trọng nhất của thuật toán Naive Bayes là ước lượng $p(x_i|c)$:

- Nếu $x_i$ là biến liên tục : giả định biến $x_i$ tuân theo phân bố _Gaussian_ và dùng tập train để ước lượng các tham số của phân bố này ($\mu$ và $\sigma$)

- Nếu $x_i$ là biến rời rạc (biến phân loại) dùng tần số (số lần $x_i$ nhận giá trị $x_i$ có nhãn $c$) để ước lượng xác suất $p(x_i| c)$. Để tránh tần số bằng 0 làm cho xác suất bằng 0 thì áp dụng thêm phép làm trơn ( _smoothing_ ) như _Laplacian smoothing_ .

# 3. Các loại phân phối dữ liệu

## 3.1 Gaussian Naive Bayes

Mô hình này được sử dụng chủ yếu trong loại dữ liệu mà các thành phần là các biến liên tục.

Với mỗi chiều dữ liệu $i$ và một lớp $c$, $x_i$ tuân theo một phân phối chuẩn có kỳ vọng $\mu_{ci}$ và phương sai $\sigma_{ci}^2$:
$$p(x_i|c)=p(x_i|\mu_{ci},\sigma_{ci}^2)=\frac{1}{\sqrt{2\pi\sigma_{ci}^2}}\exp\bigg(-\frac{(x_i-\mu_{ci})^2}{2\sigma_{ci}^2}\bigg) \quad (8)$$

## 3.2 Multinomial Naive Bayes

Mô hình này chủ yếu được sử dụng trong phân loại văn bản mà vectơ đặc trưng được tính bằng [Bags of Words](https://machinelearningcoban.com/general/2017/02/06/featureengineering/#bag-of-words). Lúc này mỗi văn bản sẽ được biểu diễn bằng một vectơ có độ dài $d$ là số từ trong _Bags of Words_. Giá trị của thành phần thứ $i$ trong mỗi vectơ chính là số lần từ thứ $i$ xuất hiện trong văn bản đó.

Khi đó, $p(x_i|c)$ tỉ lệ với tần suất từ thứ $i$ xuất hiện trong văn bản của class $c$. Giá trị này được tính bằng cách:
$$\lambda_{ci}=p(x_i|c)=\frac{N_{ci}}{N_c} \quad (9)$$

Với: 

$N_{ci}$ là số lần từ thứ $i$ xuất hiện trong văn bản của lớp $c$. Được tính bằng tổng của tất cả các thành phần thứ $i$ của vectơ đặc trưng ứng với lớp $c$.

$N_c$ là tổng số từ xuất hiện trong lớp $c$, bằng tổng độ dài của toàn bộ văn bản thuộc lớp $c$. $N_c=\sum_{i=1}^dN_{ci} \implies \sum_{i=1}^d\lambda_{ci}=1$ 

Nhược điểm là nếu một từ chưa bao giờ xuất hiện trong lớp $c$ thì biếu thức $(9)$ sẽ bằng $0$ làm cho biểu thức $(7)$ bằng $0$. Việc này dẫn đến kết quả không chính xác. Để giải quyết việc này, ta sử dụng kỹ thuật _Laplace smoothing_ : $$\hat \lambda_{ci}=\frac{N_{ci}+\alpha}{N_c+d\alpha} \quad (10)$$
Với $\alpha$ là một số dương, thường bằng 1 để tránh trường hợp tử số bằng 0. Mẫu được cộng với $d\alpha$ để đảm bảo $\sum_{i=1}^d\hat \lambda_{ci}=1$.

## 3.3 Bernoulli Naive Bayes

Mô hình này được áp dụng cho các loại dữ liệu mà mỗi thành phần là một giá trị nhị phân bằng 0 hoặc 1. Ví dụ: cũng với loại văn bản nhưng thay vì đếm tổng số lần xuất hiện của 1 từ trong văn bản, ta chỉ cần quan tâm từ đó có xuất hiện hay không.

Khi đó $p(x_i|c)$ được tính bằng:
$$p(x_i|c)=p(i|c)^{x_i}(1-p(i|c))^{1-x_i}$$
Với $p(i|c)$ là xác xuất từ thứ $i$ xuất hiện trong văn bản của lớp $c$.

# 4. Ví dụ phân loại hoa Iris bằng thư viện sklearn

In [1]:
# Chuẩn bị dữ liệu
from sklearn.datasets import load_iris 
iris = load_iris() 
X = iris.data 
y = iris.target

# Chia dữ liệu thành tập train và tập test
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=1) 
X[:10]

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2],
       [5.4, 3.9, 1.7, 0.4],
       [4.6, 3.4, 1.4, 0.3],
       [5. , 3.4, 1.5, 0.2],
       [4.4, 2.9, 1.4, 0.2],
       [4.9, 3.1, 1.5, 0.1]])

Như ta thấy thì dữ liệu của hoa Iris là dữ liệu liên tục, vì thế ta sẽ sử dụng phân phối **Gaussian Naive Bayes**

In [2]:
# Training
from sklearn.naive_bayes import GaussianNB 
gnb = GaussianNB() 
gnb.fit(X_train, y_train) 

GaussianNB(priors=None, var_smoothing=1e-09)

In [3]:
# Dự đoán xác suất nhãn của từng điểm dữ liệu (hàng)
gnb.predict_proba(X_test[:4,:])

array([[1.00000000e+00, 4.85960219e-16, 2.24347925e-28],
       [4.81032927e-29, 9.99999988e-01, 1.16657116e-08],
       [8.55082235e-94, 9.69653014e-01, 3.03469859e-02],
       [1.00000000e+00, 1.43672766e-14, 4.85229968e-27]])

In [4]:
# Dự đoán nhãn thuộc lớp 0 hay lớp 1
gnb.predict(X_test[:4, :])

array([0, 1, 1, 0])

In [5]:
# độ chính xác của mô hình trên toàn bộ dữ liệu test
y_pred = gnb.predict(X_test)
print("Mô hình Gaussian Naive Bayes chính xác :", gnb.score(X_test,y_test)*100," %")

Mô hình Gaussian Naive Bayes chính xác : 95.0  %


# 5. Kết luận

## 5.1 Ưu điểm

Dễ sử dụng và nhanh khi cần dự đoán nhãn của dữ liệu test. Thực hiện khá tốt trong _multiclass prediction_.

Có thể hoạt động với các vectơ đặc trưng mà một phần là liên tục (sử dụng _Gaussian Naive Bayes_ ), phần còn lại ở dạng rời rạc (sử dụng _Multinomial_ hoặc _Bernoulli_ ).

Khi giả định rằng các đặc trưng của dữ liệu là độc lập với nhau thì **Naive Bayes** chạy tốt hơn các thuật toán khác như **logistic regression** khi có ít dữ liệu đào tạo.

## 5.2  Nhược điểm

Độ chính xác của **Naive Bayes** nếu so với các thuật toán khác thì không được cao.

Trong thế giới thực, hầu như bất khả thi khi các đặc trưng của dữ liệu là độc lập với nhau.

## 5.3 Ứng dụng
**Real time prediction**: Là 1 thuật toán dễ học và khá nhanh. Nó có thể được dùng để dự đoán trong thế giới thực.

**Multiclass prediction**: Có thể dự đoán xác suất của nhiều loại lớp.

**Text classification/ Spam Filtering/ Sentiment Analysis**: _Naive Bayes_ được dùng rất nhiều trong _text classification._ Tương tự nó cũng được dùng trong _Spam filtering, sentiment analysis_.

**Recommendation System**: _Naive Bayes_ và _Collaborative Filtering_ được sử dụng để xây dựng hệ thống gợi ý lọc các thông tin không được để ý và dự đoán các thông tin mà người dùng có thể quan tâm.

# 6. Tài liệu tham khảo

$^{[1]}$ [Naive Bayes - gialuan1991](https://gialuan1991.wordpress.com/2016/04/15/2-naive-bayes/)

$^{[2]}$ [Naive Bayes Classifiers - geeksforgeeks.org](https://www.geeksforgeeks.org/naive-bayes-classifiers/)

$^{[3]}$ [GaussianNB - scikit-learn.org](https://scikit-learn.org/dev/modules/generated/sklearn.naive_bayes.GaussianNB.html)

$^{[4]}$ [Naive Bayes Classifier - machinelearningcoban.com](https://machinelearningcoban.com/2017/08/08/nbc/)

$^{[5]}$ [Naive Bayes Classifier From Scratch in Python - machinelearningmastery.com](https://machinelearningmastery.com/naive-bayes-classifier-scratch-python/)

$^{[6]}$ [Định lý Bayes - wikipedia.org](https://vi.wikipedia.org/wiki/%C4%90%E1%BB%8Bnh_l%C3%BD_Bayes)